# Отток клиентов

Анализ оттока банковских клиентов
Постройте модель с предельно большим значением *F1*-меры > 0.59.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

#### Признаки

    RowNumber — индекс строки в данных
    CustomerId — id клиента
    Surname — фамилия
    CreditScore — кредитный рейтинг
    Geography — страна
    Gender — пол
    Age — возраст
    Tenure — недвижимость клиента
    Balance — баланс на счёте
    NumOfProducts — количество продуктов банка, используемых клиентом
    HasCrCard — наличие кредитной карты
    IsActiveMember — активность клиента
    EstimatedSalary — предполагаемая зарплата
    
#### Целевой признак

    Exited — факт ухода клиента

# 1. Подготовка данных

In [23]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None
import numpy as np

In [24]:
#Загружаем датасет
data = pd.read_csv('Churn.csv')
data.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1


In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [26]:
def first_view(df):
    """
    Функция проходится по столбцам df и считает количество пропущенных, нулевых и отрицательных значений
    """
    for column in df.columns:
        na = df[column].isna().sum()
        #В столбцах могут быть текстовые значения, поэтому используем try except
        try: 
            zero = df[df[column] == 0][column].count()
            negative = df[df[column] < 0][column].count()
        except TypeError:
            zero = 0
            negative = 0
            
        
        if na>0 or zero>0 or negative>0:
            print()
            print('********В столбце', column, 'обнаружено: ********')
        if na > 0:
            print(na, 'пропущеных значений')
        if zero > 0:
            print(zero, 'нулевых значений.')
        if negative >0:
            print(negative, 'отрицательных значений')

In [27]:
first_view(data)


********В столбце Tenure обнаружено: ********
909 пропущеных значений
382 нулевых значений.

********В столбце Balance обнаружено: ********
3617 нулевых значений.

********В столбце HasCrCard обнаружено: ********
2945 нулевых значений.

********В столбце IsActiveMember обнаружено: ********
4849 нулевых значений.

********В столбце Exited обнаружено: ********
7963 нулевых значений.


####  Датасете имеются пропущенные значения в столбце Tenure

In [28]:
#Приведем названия столбцов к нижнему регистру
data = data.set_axis(data.columns.str.lower(), axis='columns')

In [29]:
data.head(3)

,rownumber,customerid,surname,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1


#### Для заполнения пропусков, так как, по моему, нету связи между значениями столбца Tenure и значениями в остальных столбцах, можно воспользоваться средней величиной или медианой, но тогда частоты сместятся к средним или медианным значениям. Я заполню пропуски случайными величинами, зависящими от частоты того или иного значения столбца в непропущенных данных.

In [30]:
#Определим список уникальных значений столбца Tenure
data['tenure'].value_counts()

1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: tenure, dtype: int64

In [31]:
#Создание списков значений Tenure и вероятности того или иного значения
prob_tenure = []
tenure_list = []
for i in range(0,11):
    tenure_list.append(i)
    prob_tenure.append((data['tenure'].value_counts()[i] / (len(data) - data['tenure'].isna().sum())).round(3))

In [32]:
print(prob_tenure, tenure_list)

[0.042, 0.105, 0.104, 0.102, 0.097, 0.102, 0.097, 0.102, 0.103, 0.097, 0.049] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [33]:
#Создадим массив случайных чисел и объект Series который, в дальнейшем, присоединим к датафрейму
random_array = np.random.RandomState(12345).choice(tenure_list, size=10000, p=prob_tenure)
random_tenure = pd.Series(random_array, data.index)
random_tenure

0       9
1       3
2       2
3       2
4       6
       ..
9995    7
9996    6
9997    0
9998    3
9999    8
Length: 10000, dtype: int64

In [34]:
data['tenure_rand'] = random_tenure
data.head(3)

,rownumber,customerid,surname,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited,tenure_rand
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1,9
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0,3
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1,2


In [35]:
#Заполним пропуски в столбце Tenure и удалим вспомогательный столбец
data['tenure'].fillna(data['tenure_rand'], inplace=True)
del data['tenure_rand']
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rownumber        10000 non-null  int64  
 1   customerid       10000 non-null  int64  
 2   surname          10000 non-null  object 
 3   creditscore      10000 non-null  int64  
 4   geography        10000 non-null  object 
 5   gender           10000 non-null  object 
 6   age              10000 non-null  int64  
 7   tenure           10000 non-null  float64
 8   balance          10000 non-null  float64
 9   numofproducts    10000 non-null  int64  
 10  hascrcard        10000 non-null  int64  
 11  isactivemember   10000 non-null  int64  
 12  estimatedsalary  10000 non-null  float64
 13  exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


#### Приведем данные к лучшим типам

In [36]:
data['tenure'] = data['tenure'].astype('int8')
data['creditscore'] = data['creditscore'].astype('int16')
data['numofproducts'] = data['numofproducts'].astype('int8')
data['hascrcard'] = data['hascrcard'].astype('int8')
data['isactivemember'] = data['isactivemember'].astype('int8')
data['exited'] = data['exited'].astype('int8')
data['age'] = data['age'].astype('int8')
data['estimatedsalary'] = data['estimatedsalary'].astype('int64')
data['balance'] = data['balance'].astype('int64')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   rownumber        10000 non-null  int64 
 1   customerid       10000 non-null  int64 
 2   surname          10000 non-null  object
 3   creditscore      10000 non-null  int16 
 4   geography        10000 non-null  object
 5   gender           10000 non-null  object
 6   age              10000 non-null  int8  
 7   tenure           10000 non-null  int8  
 8   balance          10000 non-null  int64 
 9   numofproducts    10000 non-null  int8  
 10  hascrcard        10000 non-null  int8  
 11  isactivemember   10000 non-null  int8  
 12  estimatedsalary  10000 non-null  int64 
 13  exited           10000 non-null  int8  
dtypes: int16(1), int64(4), int8(6), object(3)
memory usage: 625.1+ KB


#### Подготовим данные для моделирования

#### Сначала избавимся от признаков которые точно не влияют на уход клиентов, это:
1. RowNumber
2. CustomerID
3. Surname

In [37]:
data_pre_ohe = data.drop(['rownumber', 'customerid', 'surname'] , axis=1)

#### Преобразуем категориальные признаки в численные методом OHE

In [38]:
data_ohe = pd.get_dummies(data_pre_ohe, drop_first=True)

#### Сформируем тренировочную валидационную и тестовую выборки

In [39]:
# Выделим признаки и целевой признак
target = data_ohe['exited']
features = data_ohe.drop(['exited'] , axis=1)
    
#Сформируем тестовую выборку
features_train_valid, features_test, target_train_valid, target_test = train_test_split(
    features, 
    target, 
    test_size=0.2, 
    random_state=12345)
    
#Сформируем обучающую и валидационную выборки
features_train, features_valid, target_train, target_valid = train_test_split(
    features_train_valid, 
    target_train_valid, 
    test_size=0.25, 
    random_state=12345)

#### Проведем масштабирование числовых признаков

In [40]:
#Выделим числовые признаки
numeric = ['creditscore', 'age', 'tenure', 'balance', 'numofproducts', 'estimatedsalary']
    
#Обучим объект структуры StandartScaler
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

## Вывод по шагу 1.
1. Зполнены пропуски в столбце Tenure.
2. Названия столбцов приведены к нижнему регистру.
3. Все данные приведены к нужным типам.
4. Из анализа исключены столбцы не влияющие на целевой признак.
5. Категориальные признаки приведены к численным методом OHE.
6. Выделен целевой признак - exited - факт ухода клиента.
7. Данные разбиты на тренировочную, валидационную и тестовую выборки.
8. Проведено масштабирование числовых признаков.

Данные готовы к созданию моделей.

# 2. Исследование задачи

#### Посмотрим на баланс классов

In [41]:
data['exited'].value_counts()

0    7963
1    2037
Name: exited, dtype: int64

#### Данных о уходе клиентов меньше почти в 4 раза.

#### Построим модели обучения и посмотрим на их качество без балансировки классов

In [42]:
def prediction(features_train, target_train, features_valid, target_valid, weight = None):
    """
    Функция формирует модели для обучения, подбирает необходимые параметры моделей.
    На выходе функции получим значения F1_score и AUC-ROC для Логистической регрессии
    Дерева решений и Случайного леса.
    Для дальнейшего выполнения задачи предусмотрено изменения параметра class_weight
    """
       
    #Обучим модель логистической регрессии
    model_logistic = LogisticRegression(
        random_state = 12345, 
        solver='liblinear', 
        class_weight = weight)
    
    model_logistic.fit(
        features_train, 
        target_train)
    
    #Получим предсказания для валидационной выборки
    prediction_logistic = model_logistic.predict(features_valid)
    
    #Оценим предсказания расчетом F1-score и AUC-ROC
    
    f1_logistic = f1_score(
        target_valid, 
        prediction_logistic)
    
    probabilities_valid = model_logistic.predict_proba(features_valid)
    
    probabilities_one_valid = probabilities_valid[:, 1]
    
    auc_roc_logistic = roc_auc_score(
        target_valid, 
        probabilities_one_valid)

    print('Результаты предсказания Логистической регрессии')
    print('F1_score =', f1_logistic)
    print('AUC-ROC =', auc_roc_logistic)

    #Обучим модель дерева решений, для этого сначала выбираем наилучшее значение max_depth
    depth = tree_depth(
        features_train, 
        features_valid, 
        target_train, 
        target_valid, 
        weight)
    
    model_tree = DecisionTreeClassifier(
        max_depth=depth, 
        random_state=12345, 
        class_weight = weight)
    
    model_tree.fit(features_train, target_train)
    prediction_tree = model_tree.predict(features_valid)
    
    #Оценим предсказания расчетом F1-score и AUC-ROC
    
    f1_tree = f1_score(target_valid, prediction_tree)
    
    probabilities_valid = model_tree.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    
    auc_roc_tree = roc_auc_score(
        target_valid, 
        probabilities_one_valid)
    
    print('Результаты предсказания Дерева решений')
    print('F1_score =', f1_tree)
    print('AUC-ROC =', auc_roc_tree)
    
    #Обучим модель случайного леса, для этого выберем наилучшие параметры max_depth и n_estimators
    depth, estim = forest_depth_estim(
        features_train, 
        features_valid, 
        target_train, 
        target_valid, 
        weight)
    
    model_forest = RandomForestClassifier(
        max_depth = depth, 
        n_estimators=estim, 
        random_state=12345, 
        class_weight = weight)
    
    model_forest.fit(features_train, target_train)
    prediction_forest = model_forest.predict(features_valid)
    
    #Оценим предсказания расчетом F1-score и AUC-ROC
    
    f1_forest = f1_score(target_valid, prediction_forest)
    
    probabilities_valid = model_forest.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc_forest = roc_auc_score(target_valid, probabilities_one_valid)
    
    print('Результаты предсказания Случайного леса')
    print('F1_score =', f1_forest)
    print('AUC-ROC =', auc_roc_forest)


In [43]:
def tree_depth(features_train, features_valid, target_train, target_valid, weight = None):
    """
    Функция на вход получает вылидационную и обучающие выборки а на выходе получает наилучшее значение
    max_depth, с которым F1-score максимально.
    """
    best_depth = 0
    max_f1 = 0
    
    for d in range(1,15):
        model = DecisionTreeClassifier(
            max_depth=d, 
            random_state=12345, 
            class_weight = weight)
        
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_valid)
        if max_f1 < f1_score(target_valid, predicted_valid):
            max_f1 = f1_score(target_valid, predicted_valid)
            best_depth = d
    print()
    print('Наилучшее значение max_depth =', best_depth, ', с ним f1_score =', max_f1)
    print()
    return best_depth

In [44]:
def forest_depth_estim(features_train, features_valid, target_train, target_valid, weight = None):
    """
    Функция на вход получает вылидационную и обучающие выборки а на выходе получает наилучшее значение
    max_depth и n_estimators по максимальному значению F1-score.
    """
      
    best_depth = 0
    max_f1 = 0
    best_estim = 0
    for d in range(1,15):
        for e in range(10, 100, 10):
            
            model = RandomForestClassifier(
                max_depth=d, 
                n_estimators=e, 
                random_state=12345, 
                class_weight = weight)
            
            model.fit(features_train, target_train)
            predicted_valid = model.predict(features_valid)
            if max_f1 < f1_score(target_valid, predicted_valid):
                max_f1 = f1_score(target_valid, predicted_valid)
                best_depth = d
                best_estim = e
    print()
    print('Наилучшее значение max_depth =', best_depth, 'n_estimators =', best_estim, ', с ними f1_score =', max_f1)
    print()
    return best_depth, best_estim

In [45]:
prediction(features_train, target_train, features_valid, target_valid)

Результаты предсказания Логистической регрессии
F1_score = 0.3007518796992481
AUC-ROC = 0.7703184930037084

Наилучшее значение max_depth = 7 , с ним f1_score = 0.5557299843014128

Результаты предсказания Дерева решений
F1_score = 0.5557299843014128
AUC-ROC = 0.8244529254401789

Наилучшее значение max_depth = 13 n_estimators = 30 , с ними f1_score = 0.5700636942675159

Результаты предсказания Случайного леса
F1_score = 0.5700636942675159
AUC-ROC = 0.8361216240488685


## Вывод по шагу 2.
1. Данных по ушедшим клиентам банка в 4 раза меньше чем по оставшимся.
2. Составлены модели логистической регрессии, дерева решений и случайного леса, подобраны параметры модели.
3. Для каждой модели подсчитаны F1_score и AUC_ROC. 
4. Ни одна из моделей не смогла дать предсказания с требуемым качеством f1_score = 0,59.
5. Наилучшей моделью оказалась модель случайного леса f1_score = 0,57.
6. Площадь под кривой ошибок для всех моделей больше 0,5, что говорит о адекватности моделей предсказаний, а также о том что предсказания всех трех моделей лучше предсказания случайной модели.

# 3. Борьба с дисбалансом

#### Для борьбы с дисбалансом воспользуемся тремя методами:
    1. Сбалансируем вес класса
    2. Увеличим выборку положительных объектов
    3. Уменьшим выборку отрицательных объектов

In [46]:
#Для балансировки веса встроенными в sklearn методами, добавим параметр class_weight='balanced'.
prediction(features_train, 
           target_train, 
           features_valid, 
           target_valid, 
           weight = 'balanced')

Результаты предсказания Логистической регрессии
F1_score = 0.47899910634495085
AUC-ROC = 0.7724595823683595

Наилучшее значение max_depth = 6 , с ним f1_score = 0.5587044534412956

Результаты предсказания Дерева решений
F1_score = 0.5587044534412956
AUC-ROC = 0.8090671240258203

Наилучшее значение max_depth = 8 n_estimators = 60 , с ними f1_score = 0.5990675990675991

Результаты предсказания Случайного леса
F1_score = 0.5990675990675991
AUC-ROC = 0.8533282256615997


#### Проведем увеличение тренировочной выборки

In [47]:
def upsample(features, target, repeat):
    """
    Функция увеличивает выборку объектов с положительным целевым признаком в repeat раз.
    """
    #Разделим выборку на объекты с положительным и отрицательным целевым признаком
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    # Сформируем новую тренировочную выборку
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    # Перемешаем объекты в выборке
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

#### Увеличим в 3 раза.

In [48]:
features_upsampled, target_upsampled = upsample(features_train, target_train, 3)

#### Обучим модели на новой выборке

In [49]:
prediction(
    features_upsampled, 
    target_upsampled, 
    features_valid, 
    target_valid)

Результаты предсказания Логистической регрессии
F1_score = 0.4778012684989429
AUC-ROC = 0.7721114765251089

Наилучшее значение max_depth = 6 , с ним f1_score = 0.5602605863192183

Результаты предсказания Дерева решений
F1_score = 0.5602605863192183
AUC-ROC = 0.8118217698082556

Наилучшее значение max_depth = 12 n_estimators = 70 , с ними f1_score = 0.6091503267973858

Результаты предсказания Случайного леса
F1_score = 0.6091503267973858
AUC-ROC = 0.8474883130218607


#### Обучим модели с балансировкой веса классов

In [50]:
prediction(
    features_upsampled, 
    target_upsampled, 
    features_valid, 
    target_valid, 
    weight = 'balanced')

Результаты предсказания Логистической регрессии
F1_score = 0.48035714285714287
AUC-ROC = 0.772453224270766

Наилучшее значение max_depth = 6 , с ним f1_score = 0.5587044534412956

Результаты предсказания Дерева решений
F1_score = 0.5587044534412956
AUC-ROC = 0.80906712402582

Наилучшее значение max_depth = 13 n_estimators = 90 , с ними f1_score = 0.6045918367346939

Результаты предсказания Случайного леса
F1_score = 0.6045918367346939
AUC-ROC = 0.8464535326384991


#### Проведем уменьшение тренировочной выборки

In [51]:
def downsample(features, target, fraction):
    """
    Функция уменьшает выборку объектов с отрицательным целевым признаком в fraction раз.
    """
    #Разделим выборку на объекты с положительным и отрицательным целевым признаком
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    # Сформируем новую тренировочную выборку
    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    # Перемешаем объекты в выборке
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

#### Уменьшим в 2 раза

In [52]:
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.5)

#### Обучим модели на новой выборке

In [53]:
prediction(features_downsampled, target_downsampled, features_valid, target_valid)

Результаты предсказания Логистической регрессии
F1_score = 0.490272373540856
AUC-ROC = 0.7718555630969657

Наилучшее значение max_depth = 7 , с ним f1_score = 0.5751633986928104

Результаты предсказания Дерева решений
F1_score = 0.5751633986928104
AUC-ROC = 0.8155484097603157

Наилучшее значение max_depth = 9 n_estimators = 90 , с ними f1_score = 0.6025104602510459

Результаты предсказания Случайного леса
F1_score = 0.6025104602510459
AUC-ROC = 0.854350289849774


#### Обучим модели с балансировкой веса классов

In [54]:
prediction(features_downsampled, target_downsampled, features_valid, target_valid, weight = 'balanced')

Результаты предсказания Логистической регрессии
F1_score = 0.47069271758436954
AUC-ROC = 0.772292682306527

Наилучшее значение max_depth = 6 , с ним f1_score = 0.5586272640610105

Результаты предсказания Дерева решений
F1_score = 0.5586272640610105
AUC-ROC = 0.8129455635579278

Наилучшее значение max_depth = 13 n_estimators = 70 , с ними f1_score = 0.5967940813810111

Результаты предсказания Случайного леса
F1_score = 0.5967940813810111
AUC-ROC = 0.8431608328472038


## Выводы по шагу 3.
1. Уменьшение дисбаланса классов увеличило f1_score у всех моделей.
2. Балансировка классов не помогла моделям логистической регрессии и дерева решений достичь целевого показателя f1_score = 0,59.
3. Любой из методов балансировки помогает модели случайного леса превысить требуемое значение f1_score = 0,59, при  этом разница в значениях не значительна. Наилучшее занчение f1_score = 0,609 получено при увеличении тренировочной выборки и параметрах модели - max_depth = 12 и n_estimators = 70.
4. Метрика AUC-ROC, при балансировке:
    - для логистической регрессии, почти не меняется,
    - для решающего дерева всегда меньше чем при несбалансированных классах,
    - для случайного леса всегда больше чем при несбалансированных классах.


# 4. Тестирование модели

#### Используем модель с наилучшими параметрами и  увеличенную выборку для построения финальной модели и ее тестирования.

In [55]:
model_final= RandomForestClassifier(
    max_depth = 12, 
    n_estimators=70, 
    random_state=12345)

model_final.fit(features_upsampled, target_upsampled)
prediction_valid = model_final.predict(features_valid)
prediction_test = model_final.predict(features_test)    
#Оценим предсказания расчетом F1-score и AUC-ROC на валидационной и тестовой выборках
    
f1_valid = f1_score(target_valid, prediction_valid)
f1_test = f1_score(target_test, prediction_test)    

probabilities_valid = model_final.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc_valid = roc_auc_score(target_valid, probabilities_one_valid)

probabilities_test = model_final.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
auc_roc_test = roc_auc_score(target_test, probabilities_one_test)

print('Результаты предсказания валидационной выборки')
print('F1_score =', f1_valid)
print('AUC-ROC =', auc_roc_valid)
print()
print('Результаты предсказания тестовой выборки')
print('F1_score =', f1_test)
print('AUC-ROC =', auc_roc_test)

Результаты предсказания валидационной выборки
F1_score = 0.6091503267973858
AUC-ROC = 0.8474883130218607

Результаты предсказания тестовой выборки
F1_score = 0.6347305389221557
AUC-ROC = 0.8609929563729861


## Общий вывод.
1. Проведена подкготовка данных к построению моделей.
2. Проведено исследование различных моделей на несбалансированной выборке. Наилучшей оказалась модель случайного леса с f1_score = 0,57, что меньше требуемого значения 0,59.
3. Проведена балансировка классов тремя способами:
    - балансировкой веса,
    - увеличением выборки,
    - уменьшением выборки.
4. На каждом этапе балансировки проводилось исследование качества моделей. Модель случайного леса достигала требуемого значения f1_score = 0,59 или выше при любом способое балансировки.
5. Модель с наилучшеми характеристиками была выбрана для финального тестирования на тествоой выборке. Тестирование показало:
    - на валидационной выборке F1_score = 0.609, AUC-ROC = 0.847,
    - на тестовой выборке F1_score = 0.634, AUC-ROC = 0.861.
6. Значение AUC-ROC говорят о том что полученная модель адекватна и делает предсказания лучше чем случайная.